In [6]:
# import dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import pymongo
import pprint 
from pymongo import MongoClient
from selenium import webdriver
from splinter import Browser

In [3]:
# scrape news site for title and paragraph text, save off to variables
news_url = "https://mars.nasa.gov/news/"
driver = webdriver.Firefox()
driver.get(news_url)
html = driver.page_source
news_soup = bs(html, "html.parser")

news_title = (news_soup.find("div", class_="list_text")).find("a").text
print(news_title)

news_paragraph = news_soup.find("div", class_="article_teaser_body").text
print(news_paragraph)

The MarCO Mission Comes to an End
The pair of briefcase-sized satellites made history when they sailed past Mars in 2019.


In [4]:
# find featured Mars image from JPL, save full-size image url
# note: this url doesn't seem to pull up a Mars image any longer
jpl_url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
base_jpl_url = "https://www.jpl.nasa.gov"
driver = webdriver.Firefox()
driver.get(jpl_url)
html = driver.page_source
img_soup = bs(html, "html.parser")

#build url from sytle tag
image_url  = img_soup.find("article")["style"].replace('background-image: url(','').replace(');', '')[1:-1]
featured_image_url = base_jpl_url + image_url
print(featured_image_url)
driver.close()

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA17936-1920x1200.jpg


In [8]:
# scrape latest mars weather from twitter
mars_wx_url = "https://twitter.com/MarsWxReport"
r = requests.get(mars_wx_url)
html = r.text
wx_soup = bs(html, "html.parser")

mars_weather = wx_soup.find_all("div", class_="js-tweet-text-container")[0].text
pprint.pprint(mars_weather)

('\n'
 'InSight sol 450 (2020-03-02) low -93.5ºC (-136.4ºF) high -10.4ºC (13.3ºF)\n'
 'winds from the SSW at 5.5 m/s (12.4 mph) gusting to 20.6 m/s (46.1 mph)\n'
 'pressure at 6.30 hPapic.twitter.com/82lzRqibcC\n')


In [9]:
# scrape Mars facts website, display as html table str using pandas
mars_facts_url = "https://space-facts.com/mars/"
mars_df = pd.read_html(mars_facts_url)[0]

#diplay string
mars_facts_html_str = mars_df.to_html()
print(mars_facts_html_str)

#write to html
mars_facts_html = mars_df.to_html("mars_facts.html")

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>0</th>
      <th>1</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>0</th>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <th>1</th>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <th>2</th>
      <td>Mass:</td>
      <td>6.39 × 10^23 kg (0.11 Earths)</td>
    </tr>
    <tr>
      <th>3</th>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <th>4</th>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.38 AU)</td>
    </tr>
    <tr>
      <th>5</th>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <th>6</th>
      <td>Surface Temperature:</td>
      <td>-87 to -5 °C</td>
    </tr>
    <tr>
      <th>7</th>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <th>8</th>
      <td>Recorded

In [10]:
# scrape USGS Astrogeology site for high res photos of hemispheres
usgs_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser = Browser('firefox')
browser.visit(usgs_url)

#create list of titles and urls to loop through
hemisphere_list = [(a.text, a['href']) for a in browser.find_by_css('div[class="description"] a')]

hemisphere_dict = []
for title, hemisphere_url in hemisphere_list:
    temp_dict = {}
    temp_dict["title"] = title
    browser.visit(hemisphere_url)
    temp_dict["img_url"] = browser.find_by_css('img[class="wide-image"]')['src']
    hemisphere_dict.append(temp_dict)
pprint.pprint(hemisphere_dict)
browser.quit()

[{'img_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]
